<a href="https://colab.research.google.com/github/Mahjabeenqamar1/Kaggle-Capstone-Project_Smart-Health-Wellness-Concierge/blob/main/Smart_Health_%26_Wellness_Concierge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Smart Health & Wellness Concierge Agent
Track A: Concierge Agents

Author: AI Assistant


Multi-agent system for personalized health & wellness management




In [ ]:
import os
import json
import re
import asyncio
from typing import List, Dict, Any
from datetime import datetime

# Simulate ADK imports (replace with actual imports when deploying)
try:
    from google.adk.agents import Agent, SequentialAgent
    from google.adk.models.google_llm import Gemini
    from google.adk.tools import FunctionTool, AgentTool
    from google.adk.runners import InMemoryRunner
    from google.genai import types
except ImportError:
    print("⚠️ Running in demo mode - ADK not installed")
    print("Install: pip install google-adk")

In [ ]:
# SECTION 1: Setup & Configuration
# ============================================================

def setup_environment():
    """Initialize API keys and configuration"""
    try:
        from kaggle_secrets import UserSecretsClient
        GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        print("✅ API key loaded from Kaggle Secrets")
    except:
        # Fallback for local development
        GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
        if not GOOGLE_API_KEY:
            print("🔒 Please set GOOGLE_API_KEY environment variable")
            return False

    return True

# Retry configuration for API calls
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

In [ ]:
# SECTION 2: User Profile Management
# ============================================================

class UserProfile:
    """Manages user health profile and preferences"""

    def __init__(self, profile_data: Dict[str, Any] = None):
        self.data = profile_data or self.default_profile()

    @staticmethod
    def default_profile():
        return {
            "name": "User",
            "age": 30,
            "weight_kg": 70,
            "height_cm": 170,
            "fitness_goal": "maintain weight",
            "dietary_preference": "balanced",
            "allergies": [],
            "activity_level": "moderate",
            "sleep_goal_hours": 8,
            "stress_level": "moderate"
        }

    def calculate_bmr(self):
        """Calculate Basal Metabolic Rate (Mifflin-St Jeor)"""
        w, h, a = self.data['weight_kg'], self.data['height_cm'], self.data['age']
        # Simplified formula (assuming average gender)
        bmr = 10 * w + 6.25 * h - 5 * a + 5
        return round(bmr)

    def calculate_tdee(self):
        """Total Daily Energy Expenditure"""
        bmr = self.calculate_bmr()
        multipliers = {
            "sedentary": 1.2,
            "light": 1.375,
            "moderate": 1.55,
            "active": 1.725,
            "very_active": 1.9
        }
        level = self.data.get('activity_level', 'moderate')
        return round(bmr * multipliers.get(level, 1.55))

    def to_dict(self):
        return self.data


In [ ]:
# SECTION 3: Custom Tool Functions
# ============================================================

def suggest_meal(profile_data: str, meal_type: str = "lunch") -> str:
    """
    Suggests meals based on user profile and dietary preferences

    Args:
        profile_data: JSON string of user profile
        meal_type: breakfast, lunch, dinner, or snack
    """
    profile = json.loads(profile_data)
    preference = profile.get('dietary_preference', 'balanced')

    meals = {
        "breakfast": {
            "balanced": "Oatmeal with berries, almonds, and Greek yogurt (450 cal)",
            "low_carb": "Scrambled eggs with avocado and spinach (380 cal)",
            "vegetarian": "Smoothie bowl with banana, spinach, protein powder (420 cal)"
        },
        "lunch": {
            "balanced": "Grilled chicken salad with quinoa and mixed vegetables (520 cal)",
            "low_carb": "Salmon with steamed broccoli and cauliflower rice (480 cal)",
            "vegetarian": "Lentil curry with brown rice and roasted vegetables (490 cal)"
        },
        "dinner": {
            "balanced": "Baked cod with sweet potato and green beans (510 cal)",
            "low_carb": "Beef stir-fry with zucchini noodles (460 cal)",
            "vegetarian": "Chickpea pasta with tomato sauce and roasted peppers (530 cal)"
        }
    }

    meal = meals.get(meal_type, meals["lunch"]).get(preference, "Balanced meal option")
    return f"🍽️ {meal_type.title()} suggestion: {meal}"

def recommend_workout(profile_data: str, workout_type: str = "strength") -> str:
    """
    Recommends workout based on fitness goals

    Args:
        profile_data: JSON string of user profile
        workout_type: cardio, strength, flexibility, or mixed
    """
    profile = json.loads(profile_data)
    goal = profile.get('fitness_goal', 'maintain weight')
    level = profile.get('activity_level', 'moderate')

    workouts = {
        "cardio": {
            "beginner": "20 min brisk walk + 5 min cool-down",
            "moderate": "30 min jogging + intervals + stretching",
            "advanced": "45 min HIIT workout with sprint intervals"
        },
        "strength": {
            "beginner": "Bodyweight: 3x10 squats, push-ups, planks",
            "moderate": "Gym: 3x12 deadlifts, bench press, rows",
            "advanced": "Split routine: Heavy compound lifts + accessories"
        },
        "flexibility": {
            "beginner": "15 min basic yoga flow for beginners",
            "moderate": "30 min Hatha yoga + dynamic stretching",
            "advanced": "45 min advanced yoga + deep tissue work"
        }
    }

    level_map = {"sedentary": "beginner", "light": "beginner",
                 "moderate": "moderate", "active": "advanced"}
    user_level = level_map.get(level, "moderate")

    routine = workouts.get(workout_type, workouts["strength"]).get(user_level)
    return f"💪 {workout_type.title()} workout: {routine}"

def suggest_wellness_tip(profile_data: str, focus: str = "stress") -> str:
    """
    Provides mental wellness and stress management tips

    Args:
        profile_data: JSON string of user profile
        focus: stress, sleep, mindfulness, or motivation
    """
    profile = json.loads(profile_data)
    stress_level = profile.get('stress_level', 'moderate')

    tips = {
        "stress": [
            "Try 5-minute box breathing: 4 sec in, 4 hold, 4 out, 4 hold",
            "Take a 10-minute walk outside to reset your mind",
            "Practice progressive muscle relaxation starting from toes to head"
        ],
        "sleep": [
            f"Aim for {profile.get('sleep_goal_hours', 8)} hours tonight",
            "Avoid screens 1 hour before bed - try reading instead",
            "Keep room cool (65-68°F) and completely dark"
        ],
        "mindfulness": [
            "Do a 3-minute body scan meditation right now",
            "Practice gratitude: write down 3 things you're thankful for",
            "Single-task for the next 30 minutes - no multitasking"
        ]
    }

    tip_list = tips.get(focus, tips["stress"])
    import random
    return f"🧘 Wellness tip: {random.choice(tip_list)}"

def track_sleep(hours_slept: float, quality: str = "good") -> str:
    """
    Tracks sleep data and provides feedback

    Args:
        hours_slept: Number of hours slept
        quality: poor, fair, good, excellent
    """
    feedback = []

    if hours_slept < 6:
        feedback.append("⚠️ You're sleep deprived. Try to get at least 7 hours.")
    elif hours_slept >= 7 and hours_slept <= 9:
        feedback.append("✅ Great sleep duration!")
    else:
        feedback.append("😴 You might be oversleeping. Aim for 7-9 hours.")

    quality_feedback = {
        "poor": "Consider improving sleep hygiene",
        "fair": "Room for improvement in sleep quality",
        "good": "Solid sleep quality!",
        "excellent": "Perfect sleep! Keep it up!"
    }

    feedback.append(quality_feedback.get(quality, "Sleep quality noted"))

    return f"😴 Sleep tracking: {hours_slept}h ({quality}) - {' '.join(feedback)}"


In [ ]:
# SECTION 4: Agent Definitions
# ============================================================

def create_agents():
    """Create and configure all wellness agents"""

    # Nutrition Agent
    nutrition_agent = Agent(
        name="NutritionAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="""You are a certified nutritionist and meal planning expert.
        Provide personalized meal suggestions based on user's dietary preferences,
        calorie needs, and health goals. Be specific with portions and nutritional info.""",
        tools=[FunctionTool(suggest_meal)],
        output_key="nutrition_advice"
    )

    # Fitness Agent
    fitness_agent = Agent(
        name="FitnessAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="""You are a personal trainer and fitness coach.
        Create workout plans tailored to user's fitness level and goals.
        Provide clear exercise instructions with sets, reps, and duration.""",
        tools=[FunctionTool(recommend_workout)],
        output_key="fitness_plan"
    )

    # Mental Wellness Agent
    wellness_agent = Agent(
        name="WellnessAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="""You are a mental health and wellness counselor.
        Provide stress management techniques, mindfulness practices, and
        emotional wellness support. Be empathetic and practical.""",
        tools=[FunctionTool(suggest_wellness_tip)],
        output_key="wellness_tips"
    )

    # Sleep Agent
    sleep_agent = Agent(
        name="SleepAgent",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="""You are a sleep specialist and circadian rhythm expert.
        Help users improve sleep quality with evidence-based recommendations.
        Track sleep patterns and provide actionable feedback.""",
        tools=[FunctionTool(track_sleep)],
        output_key="sleep_feedback"
    )

    # Main Coordinator Agent
    coordinator_agent = Agent(
        name="WellnessCoordinator",
        model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
        instruction="""You are a holistic wellness concierge coordinating health services.
        Understand user requests and delegate to the appropriate specialist agent.
        Provide comprehensive, personalized wellness guidance combining multiple aspects.""",
        tools=[
            AgentTool(nutrition_agent),
            AgentTool(fitness_agent),
            AgentTool(wellness_agent),
            AgentTool(sleep_agent)
        ],
        output_key="wellness_response"
    )

    # Root Sequential Agent
    root_agent = SequentialAgent(
        name="SmartWellnessConcierge",
        sub_agents=[coordinator_agent]
    )

    return root_agent

In [ ]:
# SECTION 5: Orchestration & Execution
# ============================================================

class WellnessConcierge:
    """Main orchestrator for the wellness concierge system"""

    def __init__(self, user_profile: UserProfile):
        self.profile = user_profile
        self.runner = InMemoryRunner(agent=create_agents())
        self.session_id = f"wellness_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    async def handle_request(self, user_message: str):
        """Process a single user request"""

        print(f"\n💬 User: {user_message}")

        session_state = {
            "user_profile": json.dumps(self.profile.to_dict()),
            "calories_target": self.profile.calculate_tdee(),
            "session_id": self.session_id,
            "timestamp": datetime.now().isoformat()
        }

        try:
            response = await self.runner.run_debug({
                "user_request": user_message,
                "session_state": session_state
            })

            print(f"\n🤖 Concierge: {self._extract_response(response)}")
            return response

        except Exception as e:
            print(f"❌ Error: {e}")
            return None

    def _extract_response(self, response):
        """Extract readable text from agent response"""
        if isinstance(response, list) and len(response) > 0:
            last_event = response[-1]
            if hasattr(last_event, 'content'):
                return last_event.content.parts[0].text
        return str(response)

    async def daily_wellness_flow(self):
        """Complete daily wellness routine"""

        requests = [
            "What should I eat for breakfast today?",
            "Can you suggest a 30-minute workout?",
            "I'm feeling stressed, any quick tips?",
            "I slept 7 hours last night, was that enough?"
        ]

        print(f"\n{'='*60}")
        print("🌟 Starting Daily Wellness Routine")
        print(f"{'='*60}")

        for req in requests:
            await self.handle_request(req)
            await asyncio.sleep(1)  # Brief pause between requests

        print(f"\n{'='*60}")
        print("✅ Daily Wellness Routine Complete!")
        print(f"{'='*60}")

In [ ]:
# SECTION 6: Demo & Testing
# ============================================================

def run_async(coro):
    """Safe async execution for Kaggle/Jupyter"""
    try:
        return asyncio.run(coro)
    except RuntimeError:
        import nest_asyncio
        nest_asyncio.apply()
        return asyncio.get_event_loop().run_until_complete(coro)

async def demo():
    """Demo the wellness concierge system"""

    print("\n" + "="*60)
    print("🏥 SMART HEALTH & WELLNESS CONCIERGE")
    print("="*60)

    # Create user profile
    profile_data = {
        "name": "Sarah",
        "age": 28,
        "weight_kg": 65,
        "height_cm": 165,
        "fitness_goal": "lose weight",
        "dietary_preference": "balanced",
        "activity_level": "moderate",
        "sleep_goal_hours": 8,
        "stress_level": "high"
    }

    profile = UserProfile(profile_data)

    print(f"\n📊 User Profile:")
    print(f"   Name: {profile.data['name']}")
    print(f"   BMR: {profile.calculate_bmr()} cal/day")
    print(f"   TDEE: {profile.calculate_tdee()} cal/day")
    print(f"   Goal: {profile.data['fitness_goal']}")

    # Initialize concierge
    concierge = WellnessConcierge(profile)

    # Run daily flow
    await concierge.daily_wellness_flow()

    # Custom requests
    print(f"\n{'='*60}")
    print("💡 Custom Requests")
    print(f"{'='*60}")

    await concierge.handle_request(
        "I have 20 minutes before work, what can I do?"
    )


In [ ]:
# SECTION 7: Main Execution
# ============================================================

if __name__ == "__main__":

    # Setup
    if setup_environment():
        print("\n🚀 Initializing Smart Wellness Concierge...")

        # Run demo
        run_async(demo())

        print("\n✅ Demo completed successfully!")
        print("\n💡 Next Steps:")
        print("   1. Customize user profile")
        print("   2. Add more specialized agents")
        print("   3. Integrate with health tracking APIs")
        print("   4. Deploy to Google Agent Engine")
    else:
        print("\n❌ Setup failed. Please configure API keys.")

"""
DEPLOYMENT NOTES:
================

1. Kaggle Setup:
   - Add GOOGLE_API_KEY to Kaggle Secrets
   - Install: !pip install google-adk

2. Local Development:
   - export GOOGLE_API_KEY="your_key_here"
   - pip install google-adk

3. Agent Engine Deployment:
   - Use ADK CLI: adk deploy agent_engine
   - Configure resources in agent_engine_config.json

4. Extensions:
   - Add database for user history
   - Integrate wearable device APIs
   - Add calendar scheduling
   - Build mobile app interface
"""